<a href="https://colab.research.google.com/github/donkalakamakshiharshitha/ML-DL/blob/master/Copy_of_Untitled7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
# Import the required modules and functions from Keras
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [3]:
# HYPERPARAMETERS
# Set the batch size for training
batch_size = 128

# Set the number of classes (digits 0-9)
num_classes = 10

# Set the number of training epochs
epochs = 10

# Input Image Dimensions
# Set the dimensions of the input images
img_rows, img_cols = 28, 28

# Splitting the Data
# Load the MNIST dataset and split it into training and testing sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
# Reshape the data
# Check the image data format
if K.image_data_format() == 'channels_first':
    # Reshape the training and testing data for 'channels_first' format
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    # Reshape the training and testing data for 'channels_last' format
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


In [5]:
# Convert the data type to 'float32'
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Image Normalization
# Normalize the pixel values to the range [0, 1]
x_train /= 255
x_test /= 255

# One Hot Encoding
# Convert the class labels to one-hot encoded vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
# KERAS MODEL

model = Sequential()

# First Layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

# Hidden Layers (Convolution)
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))

# Pooling Layer
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

# DNN Classifier Layers
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(num_classes, activation='softmax'))


In [ ]:
# Compiling the Model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.RMSprop(), metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('\nTest loss:', score[0])
print('\nTest accuracy:', score[1])


Epoch 1/10
 25/469 [>.............................] - ETA: 8:11 - loss: 0.0360 - accuracy: 0.9928

In [ ]:
model.save('cnn-mnist-model.h5')

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import tensorflow as tf
import cv2


title = "Mnist Digit classifier"

In [ ]:
img_size = 28
labels = ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]

In [ ]:
model = tf.keras.models.load_model("/content/cnn-mnist-model.h5")

In [ ]:
def predict(img):

  img = cv2.resize(img, (img_size, img_size))
  img = img.reshape(1, img_size, img_size, 1)

  preds = model.predict(img)[0]

  return {label: float(pred) for label, pred in zip(labels, preds)}

label = gr.outputs.Label(num_top_classes=3)

In [ ]:
interface = gr.Interface(fn=predict, inputs="sketchpad", outputs=label, title=title)
interface.launch(debug=True)